In [1]:
# !pip install statsmodels 

## 公共板块

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def print_variable(model, X):
    #### 模型系数的基本输出
    df_varable1 = pd.DataFrame(model.params)
    df_varable1.columns = ['系数']
    R_mcfadden = 1 - (model.llf / model.llnull)  ## LRI
    df_standparams = pd.DataFrame(model.params * np.std(X) * np.sqrt(R_mcfadden) / np.std(
        np.log(model.fittedvalues / (1 - model.fittedvalues))))  ## 使用LRI指标
    df_standparams.columns = ['标准化系数']
    df_varable = pd.concat([df_varable1, df_standparams], axis=1)
    df_varable2 = pd.DataFrame(model.bse)
    df_varable2.columns = ['标准误']
    df_varable = pd.concat([df_varable, df_varable2], axis=1)
    df_varable3 = model.wald_test_terms()
    df_varable3 = df_varable3.summary_frame()
    df_varable3.columns = ['Wald检验', 'P0值', '自由度']
    df_varable = pd.concat([df_varable, df_varable3], axis=1)
    df_varable['P值'] = df_varable3['P0值']
    del df_varable['P0值']
    df_varable4 = pd.DataFrame(np.exp(model.params))
    df_varable4.columns = ['OR值']
    df_varable = pd.concat([df_varable, df_varable4], axis=1)
    df_varable6 = pd.DataFrame(np.exp(model.conf_int()))
    df_varable6.columns = ['OR的 95% 置信区间下限', 'OR的 95% 置信区间上限']
    df_varable = pd.concat([df_varable, df_varable6], axis=1)
    
    return df_varable

## 读取数据

1) csv 数据读取格式：

pd.read_csv("xxx.csv")

pd.read_csv("xxx.csv", encoding='gb18030')

pd.read_csv("xxx.csv", encoding='utf-8')

如有需要，添加参数 engine='python'

2) xlsx读取格式：

pd.read_excel("xxx.xlsx")

In [3]:
df = pd.read_excel('test_data.xlsx')
df.head()

,Unnamed: 0,性别,年龄,日均费用,入院方式,咳嗽,咳嗽时间年,咳痰,咳痰时间年,胸闷,...,血小板,红细胞,白细胞,PH值测定,二氧化碳分压测定,氧分压测定,血沉,肌酐,尿素氮,D二聚体
0,0,男,79.0,1587.19,门诊,有,10.0,有,10.0,无,...,162,4.45,7.70,7.361,59.0,74.2,7,80.5,4.7,604
1,1,男,NaN,5696.54,NaN,NaN,40.0,无,40.0,无,...,186,3.40,6.00,7.298,62.7,192.3,62,122.0,8.4,864
2,2,女,71.0,1911.30,NaN,NaN,NaN,无,0.0,无,...,196,3.51,9.00,7.306,65.0,191.0,77,63.0,5.7,988
3,3,女,71.0,1808.35,NaN,NaN,3.0,无,3.0,无,...,205,5.22,15.60,7.413,38.1,90.4,56,68.0,5.0,399
4,4,男,87.0,2032.80,NaN,NaN,NaN,有,30.0,无,...,424,4.25,11.93,7.416,37.0,59.4,52,109.0,7.9,791


In [4]:
df.columns

Index(['Unnamed: 0', '性别', '年龄', '日均费用', '入院方式', '咳嗽', '咳嗽时间年', '咳痰', '咳痰时间年',
       '胸闷', '胸闷时间年', '喘息', '喘息时间年', '气促', '气促时间年', '住院天数', '急性加重再入院天数',
       '一年内是否急性加重再入院', '转归', 'B型钠尿肽前体', '嗜酸性粒细胞数', '嗜酸性粒细胞比率', '中性粒细胞数',
       '淋巴细胞数', '血红蛋白', '血小板', '红细胞', '白细胞', 'PH值测定', '二氧化碳分压测定', '氧分压测定',
       '血沉', '肌酐', '尿素氮', 'D二聚体'],
      dtype='object')

### 以下结果中，variables为筛选出来的变量，df_params为变量筛选的过程

## 前向

In [5]:
def forward_selected(df, exog, endog, threshold_in=0.05, noconstant=False):
    remaining = exog.copy()
    X = df[endog]
    y = df[endog]
    selected = []
    if noconstant == False:
        X_NEW = sm.add_constant(X)
        del X_NEW[endog]
        try:
            model = sm.GLM(y, X_NEW, family=sm.families.Binomial()).fit()
        except PerfectSeparationError:
            raise LogisticRegCustomError('【拟合结果不可用，请重新选择变量或者方法】')
        current_score, best_new_score = model.llf, model.llf
    else:
        current_score, best_new_score = -1e12, -1e12

    i = 0 
    df_params =  pd.DataFrame(columns=['模型', '变量', '系数', '标准化系数', '标准误', 'Wald检验', '自由度', 'P值', 'OR值', 'OR的 95% 置信区间下限', 'OR的 95% 置信区间上限'])

    while remaining and current_score == best_new_score:
        i = i + 1
        scores_with_candidates = []
        for candidate in remaining:
            ## 逐个加入变量，找出R方最高的变量
            exog = selected + [candidate]
            X = df[exog] 
            if noconstant == False:
                X = sm.add_constant(X)
            model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
            scores_with_candidates.append((model.llf, candidate, model.pvalues[candidate]))
        scores_with_candidates.sort()
        best_new_score, best_candidate, current_p_value = scores_with_candidates.pop()

        if current_score < best_new_score and  current_p_value < threshold_in:
            ## 对比当前加入的变量的模型分数与最好的分数，如果最好的分数不改变，则最好的变量就是上一次中选择的变量
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
            if selected ==[]:
                print('警告：模型未能进入任何变量, 请重新选择变量筛选的方法')
            else:
                X = df[selected]
                if noconstant == False:
                    X = sm.add_constant(X)
                model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
                df_params0 = print_variable(model, sm.add_constant(df[selected]))
                df_params0 = df_params0.reset_index().rename(index=str, columns={'index':'变量'})
                df_params0.insert(0, '模型', [i]+['']*(len(model.params)-1))
                df_params = pd.concat([df_params, df_params0], axis=0)
    exog = selected

    return exog, df_params

参数

exog： 自变量

endog： 因变量

threshold_in：当变量p值小于threshold_in时，才能进入模型

noconstant：模型不添加截距（不用修改这个参数）

In [6]:
exog = ['淋巴细胞数', '血红蛋白', '血小板', '红细胞', '白细胞', 'PH值测定', '二氧化碳分压测定', '氧分压测定',
       '血沉', '肌酐', '尿素氮', 'D二聚体']
endog = '一年内是否急性加重再入院'

threshold_in = 0.05 

In [7]:
variables, df_params = forward_selected(df, exog, endog, threshold_in=threshold_in, noconstant=False)

In [8]:
variables

['二氧化碳分压测定', '淋巴细胞数', '尿素氮']

In [9]:
df_params

,模型,变量,系数,标准化系数,标准误,Wald检验,自由度,P值,OR值,OR的 95% 置信区间下限,OR的 95% 置信区间上限
0,1,const,-4.423606,-0.000000,0.519632,72.470551,1,1.695432e-17,0.011991,0.004331,0.033202
1,,二氧化碳分压测定,0.035752,0.203088,0.009134,15.319781,1,9.076113e-05,1.036399,1.018009,1.055120
0,2,const,-3.567144,-0.000000,0.636836,31.375155,1,2.126852e-08,0.028236,0.008105,0.098375
1,,二氧化碳分压测定,0.031410,0.157660,0.009469,11.003301,1,9.094977e-04,1.031908,1.012934,1.051238
2,,淋巴细胞数,-0.593803,-0.160093,0.268974,4.873756,1,2.726808e-02,0.552223,0.325959,0.935547
0,3,const,-4.128293,-0.000000,0.705430,34.247813,1,4.852203e-09,0.016110,0.004042,0.064205
1,,二氧化碳分压测定,0.033565,0.179032,0.009563,12.318798,1,4.484195e-04,1.034135,1.014932,1.053702
2,,淋巴细胞数,-0.502971,-0.144098,0.267611,3.532453,1,6.017869e-02,0.604731,0.357908,1.021771
3,,尿素氮,0.054001,0.085803,0.026733,4.080454,1,4.338200e-02,1.055485,1.001607,1.112262


## 后向

In [10]:
def backward_selected(df, exog, endog, threshold_out=0.1, noconstant=False):
    y = df[endog]
    X = df[exog]
    remaining = exog.copy()
    if noconstant == False:
        X = sm.add_constant(X)
    model= sm.GLM(y, X, family=sm.families.Binomial()).fit()
    current_score, best_new_score = model.llf, model.llf
        
    i = 1
    df_params =  pd.DataFrame(columns=['模型', '变量', '系数', '标准化系数', '标准误', 'Wald检验', '自由度', 'P值', 'OR值', 'OR的 95% 置信区间下限', 'OR的 95% 置信区间上限'])
    df_params0 = print_variable(model, sm.add_constant(df[remaining]))
    df_params0 = df_params0.reset_index().rename(index=str, columns={'index':'变量'})
    df_params0.insert(0, '模型', [i]+['']*(len(model.params)-1))
    df_params = pd.concat([df_params, df_params0], axis=0)

    ## while remaining:
    while remaining and current_score == best_new_score:
        i  = i + 1
        scores_with_candidates = []
        for candidate in remaining:
            p_value = model.pvalues[candidate]
            selected_remove = remaining.copy()
            selected_remove.remove(candidate)
            exog = selected_remove
            X = df[exog]
            if noconstant == False:
                X = sm.add_constant(X)
            model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
            scores_with_candidates.append((model.llf, selected_remove, candidate, p_value))
        scores_with_candidates.sort()
        best_new_score, best_selected, worst_candidate, current_p_value = scores_with_candidates.pop()

        if best_new_score < current_score and threshold_out < current_p_value:
            remaining = best_selected
            current_score = best_new_score
            
            if remaining == []:
                raise LogisticRegCustomError('警告：模型未能进入任何变量, 请重新选择变量筛选的方法')
            X = df[remaining]
            if noconstant == False:
                X = sm.add_constant(X)
            model = sm.GLM(y, X, family=sm.families.Binomial()).fit()

            df_params0 = print_variable(model, sm.add_constant(df[remaining]))
            df_params0 = df_params0.reset_index().rename(index=str, columns={'index':'变量'})
            df_params0.insert(0, '模型', [i]+['']*(len(model.params)-1))
            df_params = pd.concat([df_params, df_params0], axis=0)
            
            if max(model.pvalues[1:]) < threshold_out:
                break
            
    exog = remaining

    return exog, df_params


In [11]:
exog = ['淋巴细胞数', '血红蛋白', '血小板', '红细胞', '白细胞', 'PH值测定', '二氧化碳分压测定', '氧分压测定',
       '血沉', '肌酐', '尿素氮', 'D二聚体']
endog = '一年内是否急性加重再入院'

threshold_out = 0.1

参数

exog： 自变量

endog： 因变量

threshold_out：当变量p值大于threshold_in时，去掉该变量

noconstant：模型不添加截距（不用修改这个参数）

In [12]:
variables, df_params = backward_selected(df, exog, endog, threshold_out=threshold_out, noconstant=False)

In [13]:
variables

['淋巴细胞数', '二氧化碳分压测定', '尿素氮']

In [14]:
df_params

,模型,变量,系数,标准化系数,标准误,Wald检验,自由度,P值,OR值,OR的 95% 置信区间下限,OR的 95% 置信区间上限
0,1,const,13.428956,0.000000,30.307608,0.196328,1,6.577016e-01,679393.711890,1.082028e-20,4.265840e+31
1,,淋巴细胞数,-0.496894,-0.139111,0.273426,3.302531,1,6.917323e-02,0.608418,3.560090e-01,1.039783e+00
2,,血红蛋白,-0.020695,-0.152364,0.010527,3.864831,1,4.930817e-02,0.979517,9.595144e-01,9.999373e-01
3,,血小板,-0.000998,-0.035343,0.002174,0.210581,1,6.463123e-01,0.999003,9.947541e-01,1.003269e+00
4,,红细胞,0.265556,0.072420,0.272024,0.953010,1,3.289542e-01,1.304156,7.652128e-01,2.222679e+00
...,...,...,...,...,...,...,...,...,...,...,...
4,,D二聚体,-0.000143,-0.089032,0.000128,1.248885,1,2.637656e-01,0.999857,9.996059e-01,1.000108e+00
0,10,const,-4.128293,-0.000000,0.705430,34.247813,1,4.852203e-09,0.016110,4.042424e-03,6.420494e-02
1,,淋巴细胞数,-0.502971,-0.144098,0.267611,3.532453,1,6.017869e-02,0.604731,3.579079e-01,1.021771e+00
2,,二氧化碳分压测定,0.033565,0.179032,0.009563,12.318798,1,4.484195e-04,1.034135,1.014932e+00,1.053702e+00


In [15]:
df_params.iloc[50:, :]

,模型,变量,系数,标准化系数,标准误,Wald检验,自由度,P值,OR值,OR的 95% 置信区间下限,OR的 95% 置信区间上限
4,,红细胞,0.294467,0.079827,0.253026,1.354387,1,2.445127e-01,1.342410,0.817541,2.204251
5,,二氧化碳分压测定,0.031472,0.163062,0.010244,9.438899,1,2.124310e-03,1.031972,1.011459,1.052901
6,,肌酐,-0.003663,-0.061169,0.004918,0.554732,1,4.563905e-01,0.996344,0.986786,1.005994
7,,尿素氮,0.078586,0.121295,0.040752,3.718808,1,5.380275e-02,1.081757,0.998715,1.171702
8,,D二聚体,-0.000167,-0.104300,0.000133,1.576464,1,2.092707e-01,0.999833,0.999571,1.000094
0,6,const,-2.607193,-0.000000,1.362770,3.660169,1,5.572797e-02,0.073741,0.005102,1.065866
1,,淋巴细胞数,-0.515822,-0.143217,0.269213,3.671210,1,5.535998e-02,0.597009,0.352230,1.011895
2,,血红蛋白,-0.019362,-0.141367,0.010090,3.682201,1,5.499620e-02,0.980825,0.961618,1.000414
3,,红细胞,0.289325,0.078250,0.254632,1.291059,1,2.558529e-01,1.335526,0.810791,2.199864
4,,二氧化碳分压测定,0.032504,0.168020,0.010069,10.421150,1,1.245803e-03,1.033039,1.012852,1.053628


## 双向选择

In [16]:
## https://www.jianshu.com/p/744032f2f722
def stepwise_selected(df, exog, endog, threshold_in=0.1, threshold_out=0.11, noconstant=False):
    exog_list = exog.copy()
    X = df[exog]
    y = df[endog]
    selected = []
    
    i = 0
    df_params =  pd.DataFrame(columns=['模型', '变量', '系数', '标准化系数', '标准误', 'Wald检验', '自由度', 'P值', 'OR值', 'OR的 95% 置信区间下限', 'OR的 95% 置信区间上限'])
 
    while True:
        i = i + 1
        changed = False
        # 前向
        remaining = list(set(exog_list) - set(selected))
        new_pval = pd.Series(index=remaining)
        for candidate in remaining:
            X = df[selected + [candidate]]
            if noconstant == False:
                X = sm.add_constant(X)
            model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
            new_pval[candidate] = model.pvalues[candidate]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            selected.append(best_feature)
            changed = True

        # 后向
        exog = selected
        X = df[exog]
        if noconstant == False:
            X = sm.add_constant(X)
        model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
            
        pvalues = model.pvalues.iloc[1:]  ## 使用每个系数的p值, 不需要常数的p值 
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            selected.remove(worst_feature)
            changed = True
        if not changed:
            break

        if changed == True:
            X = df[selected]
            if noconstant == False:
                X = sm.add_constant(X)
            model = sm.GLM(y, X, family=sm.families.Binomial()).fit()

            df_params0 = print_variable(model, sm.add_constant(X))
            df_params0 = df_params0.reset_index().rename(index=str, columns={'index':'变量'})
            df_params0.insert(0, '模型', [i]+['']*(len(model.params)-1))
            df_params = pd.concat([df_params, df_params0], axis=0)

    exog = selected

    return exog, df_params

参数

exog： 自变量

endog： 因变量

threshold_in：当变量p值小于threshold_in时，选择该变量

threshold_out：当变量p值大于threshold_out时，去掉该变量

noconstant：模型不添加截距（不用修改这个参数）

In [17]:
exog = ['淋巴细胞数', '血红蛋白', '血小板', '红细胞', '白细胞', 'PH值测定', '二氧化碳分压测定', '氧分压测定',
       '血沉', '肌酐', '尿素氮', 'D二聚体']
endog = '一年内是否急性加重再入院'

threshold_in=0.05
threshold_out=0.1

In [18]:
variables, df_params = stepwise_selected(df, exog, endog, threshold_in=threshold_in, threshold_out=threshold_out, noconstant=False)

C:\Users\Acer\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [19]:
variables

['二氧化碳分压测定', '尿素氮']

In [20]:
df_params

,模型,变量,系数,标准化系数,标准误,Wald检验,自由度,P值,OR值,OR的 95% 置信区间下限,OR的 95% 置信区间上限
0,1,const,-4.423606,-0.000000,0.519632,72.470551,1,1.695432e-17,0.011991,0.004331,0.033202
1,,二氧化碳分压测定,0.035752,0.203088,0.009134,15.319781,1,9.076113e-05,1.036399,1.018009,1.055120
0,2,const,-4.944359,-0.000000,0.576422,73.576388,1,9.681592e-18,0.007123,0.002302,0.022047
1,,二氧化碳分压测定,0.037595,0.218852,0.009238,16.561231,1,4.710419e-05,1.038310,1.019680,1.057282
2,,尿素氮,0.064952,0.112638,0.026677,5.928061,1,1.490163e-02,1.067108,1.012747,1.124388
